## Подсчет профиля города

### Содержание:
* Подгрузка выборки
* Подсчет количества по годам
* Подсчет лидеров заявителей города
* [Подсчет количества по областям с fractional count](#second-bullet)
* Подсчет заявителей по областям с $ИТС \geq 1$

In [42]:
import pandas as pd
from glob import glob
from collections import defaultdict
import numpy as np

Подгрузим загруженную с patstat выборку

In [43]:
CITY_NAME = 'токио_fua'

In [44]:
df = pd.DataFrame([])
for indx, path in enumerate((glob('{}\\resulttable-*\\*'.format(CITY_NAME)))):
    print(f'Количество заявок в архиве #{indx}:', pd.read_table(path, sep=';')['appln_id'].nunique())
    df = pd.concat([df, pd.read_table(path, sep=';', na_values='hue')])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.


Количество заявок в архиве #0: 2429
Количество заявок в архиве #1: 6029
Количество заявок в архиве #2: 6243
Количество заявок в архиве #3: 6039
Количество заявок в архиве #4: 6060
Количество заявок в архиве #5: 5827
Количество заявок в архиве #6: 6162
Количество заявок в архиве #7: 5666
Количество заявок в архиве #8: 6550
Количество заявок в архиве #9: 5995
Количество заявок в архиве #10: 6258
Количество заявок в архиве #11: 6070
Количество заявок в архиве #12: 6490
Количество заявок в архиве #13: 6324
Количество заявок в архиве #14: 5808
Количество заявок в архиве #15: 6105
Количество заявок в архиве #16: 6227
Количество заявок в архиве #17: 6306
Количество заявок в архиве #18: 6272
Количество заявок в архиве #19: 6301
Количество заявок в архиве #20: 6312
Количество заявок в архиве #21: 6535
Количество заявок в архиве #22: 7041
Количество заявок в архиве #23: 6192
Количество заявок в архиве #24: 5623
Количество заявок в архиве #25: 6380
Количество заявок в архиве #26: 6418
Количество 

In [45]:
df.head()

,appln_id,appln_filing_year,appln_auth,techn_field_nr,person_id,person_ctry_code,person_name,person_address,applt_seq_nr
0,425993975,2014,EP,20,6610,JP,NTN Corporation,"3-17, Kyomachibori 1-chome, Nishi-ku,Osaka-shi...",1
1,425993975,2014,EP,31,6610,JP,NTN Corporation,"3-17, Kyomachibori 1-chome, Nishi-ku,Osaka-shi...",1
2,423224692,2014,EP,31,6610,JP,NTN Corporation,"3-17, Kyomachibori 1-chome, Nishi-ku,Osaka-shi...",1
3,473040864,2016,EP,1,7673,JP,"Dai Nippon Printing Co., Ltd.","1-1-1, Ichigaya-Kagacho, Shinjuku-ku,Tokyo 162...",1
4,473040864,2016,EP,9,7673,JP,"Dai Nippon Printing Co., Ltd.","1-1-1, Ichigaya-Kagacho, Shinjuku-ku,Tokyo 162...",1


In [46]:
df.shape

(423631, 9)

In [47]:
df.appln_id.nunique()

238991

In [48]:
df.shape

(423631, 9)

In [49]:
df.appln_id.nunique()

238991

**Посчитаем количество заявок города по годам**

In [52]:
pd.DataFrame(df.groupby('appln_filing_year').nunique()['appln_id']).T

appln_filing_year,2014,2015,2016,2017
appln_id,60342,60922,56718,35923


**Посчитаем топ заявителей по городу**
* Нужно оставить тех, кто входит в первые 2 квартиля

In [53]:
%%time
current_aplicant = []
for indx, person_name in enumerate(df.person_name.unique()):
    #df[df.person_name == person_name].groupby(['appln_filing_year', 'appln_id']).nunique()['appln_id'].groupby('appln_filing_year').size()
    aux_ = pd.DataFrame(df[df.person_name == person_name].groupby(['appln_filing_year', 'appln_id']).nunique()['appln_id'].groupby('appln_filing_year').size()).T
    #aux_.index = [person_name]
    #aux_ = pd.DataFrame(dict(aux_)).reset_index()
    current_aplicant.append([])
    current_aplicant[indx].append(person_name)
    for year in [2014, 2015, 2016, 2017]:
        try:
            current_aplicant[indx].append(aux_[year][0])
        except KeyError:
            current_aplicant[indx].append(0)

Wall time: 2min 37s


In [54]:
df_applicants_leaders = pd.DataFrame(current_aplicant, columns=['person_name', 2014, 2015, 2016, 2017]).assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
df_applicants_leaders.columns = df_applicants_leaders.columns.tolist()[0:-1] + ['2014-16']

In [55]:
df_applicants_leaders

,person_name,2014,2015,2016,2017,2014-16
0,"Dai Nippon Printing Co., Ltd.",133,133,158,40,424
1,Keihin Corporation,22,36,19,15,77
2,Canon Kabushiki Kaisha,3979,4100,3941,3422,12020
3,"Sumitomo Metal Mining Co., Ltd.",31,69,44,18,144
4,FUJITSU LIMITED,2256,1925,1954,1775,6135
5,"Ricoh Company, Ltd.",1541,1553,1516,816,4610
6,National University Corporation Tokyo Universi...,2,9,2,0,13
7,FANUC CORPORATION,32,63,93,82,188
8,"NIPPON SODA CO., LTD.",16,10,19,3,45
9,"Mitsubishi Gas Chemical Company, Inc.",138,91,116,16,345


In [56]:
quantiles = df_applicants_leaders['2014-16'].quantile([0.25,0.5,0.75])
quantiles

0.25    1.0
0.50    1.0
0.75    4.0
Name: 2014-16, dtype: float64

In [57]:
df_to_save = df_applicants_leaders[df_applicants_leaders['2014-16'] >= quantiles[0.5]].sort_values('2014-16', \
                                                    ascending=False)
df_to_save.to_excel(f'{CITY_NAME}\\{CITY_NAME}_лидеры.xlsx', index=False)

In [58]:
df[['appln_id', 'appln_filing_year']].nunique()

appln_id             213905
appln_filing_year         4
dtype: int64

**Посчитаем значения по областям с учетом fractional_count** <a class="anchor" id="second-bullet"></a>

> **структура словаря**:
> ```
>    Область
>        Год i
>            Число заявок за год i с учетом fractional_count
>            
>Пример
>    Область
>        2014
>            Число заявок за год i с учетом fractional_count
>        2015
>            Число заявок за год i с учетом fractional_count
>        2016
>            Число заявок за год i с учетом fractional_count
>        2017
>            Число заявок за год i с учетом fractional_count
>```

In [59]:
'''
    На вход:
        сырая табличка заявок
    На выход: словарь структуры выше, содержащий в себе количество заявок по областям по годам 
            с учетом fractional count
    
    dict_of_tech_fileds_per_city :dict: словарь для счёта по тех.областям
    tech_fileds_column :pd Series: колонка для хранения тех.областей
    appln_year :np array: (фактически: единичной длины) 
    tech_fileds_list :np array: лист со всеми (уникальными) тех.бластями заявки, 
                                длина листа равна количеству областей у заявки 
    fractional_count_per_application :float: число с плавающей точкой, равное доле, 
    которую нужно прибавить к каждой области по итогами анализа текущей заявки
    1/количество тех.областей = dfrac{1}{len(tech_fileds_list)}
'''

# создадим словарь структуры, описанной выше
dict_of_tech_fileds_per_city = defaultdict(lambda: defaultdict(np.float16))

# пройдёмся по всем уникальным заявкам в цикле и возьмём
#    А) Колонку с тех областями для каждой заявки
#    Б) Год каждой заявки
for appln_id in df['appln_id'].unique():
    tech_fileds_column = df[df['appln_id'] == appln_id].techn_field_nr
    appln_year = df[df['appln_id'] == appln_id].appln_filing_year.unique()
    # если внезапно было больше одного года для заявки, залогируем это выводом года. пока такого не было
    if len(appln_year) != 1:       
        print(appln_year)
    tech_fileds_list = tech_fileds_column.unique()
    fractional_count_per_application = 1 / len(tech_fileds_list)
    
    # добавим в словарь к каждой встретившеся в заявке области долю fractional_count по итогам анализа заявки
    for tech_filed in tech_fileds_list:
        dict_of_tech_fileds_per_city[tech_filed][appln_year[0]] += fractional_count_per_application

In [60]:
df_techs = pd.DataFrame(dict_of_tech_fileds_per_city).T.sort_index().replace(np.nan, 0)
df_techs = df_techs.assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
df_techs.columns = df_techs.columns.tolist()[0:-1] + ['2014-16']
df_techs.head()

,2014,2015,2016,2017,2014-16
1,5199.898810,4984.857143,4222.973810,3002.635714,14407.729762
2,3856.600000,3521.471429,3095.644048,2246.111905,10473.715476
3,1912.908333,2027.583333,2100.966667,1463.266667,6041.458333
4,3519.558333,3263.533333,3290.316667,1661.733333,10073.408333
5,752.726190,665.769048,608.169048,445.292857,2026.664286


In [61]:
df_techs

,2014,2015,2016,2017,2014-16
1,5199.898810,4984.857143,4222.973810,3002.635714,14407.729762
2,3856.600000,3521.471429,3095.644048,2246.111905,10473.715476
3,1912.908333,2027.583333,2100.966667,1463.266667,6041.458333
4,3519.558333,3263.533333,3290.316667,1661.733333,10073.408333
5,752.726190,665.769048,608.169048,445.292857,2026.664286
6,6687.655952,6400.826190,5856.516667,4411.166667,18944.998810
7,677.810714,607.450000,646.150000,398.266667,1931.410714
8,5030.522619,5355.857143,4921.130952,3862.221429,15307.510714
9,4730.294048,4946.876190,4880.411905,3572.238095,14557.582143
10,2635.311905,2736.792857,2551.476190,1572.016667,7923.580952


Получим заявителей по областям с $ИТС >= 1.1$
* Нужно убрать хвосты, равные _менее_ $5$ заявок за $2014-2016$ годы

In [62]:
fields_with_its_greater_or_equal1and1 = '''1
3
4
6
8
9
10
12
13
23
28
29
31
32'''
fields_with_its_greater_or_equal1and1 = fields_with_its_greater_or_equal1and1.split('\n')
print('string of fileds for sql: (', ', '.join(fields_with_its_greater_or_equal1and1), ')')
fields_with_its_greater_or_equal1and1 = tuple(map(int, fields_with_its_greater_or_equal1and1))

string of fileds for sql: ( 1, 3, 4, 6, 8, 9, 10, 12, 13, 23, 28, 29, 31, 32 )


In [63]:
%%time
# создадим словарь структуры, описанной выше
dict_of_leaders_per_tech_fileds = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for appln_id in df['appln_id'].unique():
    
    current_app = df[df['appln_id'] == appln_id][['appln_filing_year', 'techn_field_nr', 'person_name']].drop_duplicates()
#     curr_app_year = current_app.appln_filing_year.unique()
#     curr_app_fields = current_app.techn_field_nr.unique()
    
    for row in current_app.values:
        curr_app_year, tech_filed, person_name = row
        if tech_filed in fields_with_its_greater_or_equal1and1:
            dict_of_leaders_per_tech_fileds[tech_filed][curr_app_year][person_name] += 1

Wall time: 9min 48s


In [64]:
%%time
df_leaders_per_tech_fileds = pd.DataFrame([])
flatten_of_leaders_per_tech_fileds = []
for key, val in dict_of_leaders_per_tech_fileds.items():
    cur_field = pd.DataFrame(val).replace(np.nan, 0)
    cur_field = cur_field.reindex(columns=[2014, 2015, 2016, 2017])
    cur_field = cur_field.assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
    cur_field.columns = cur_field.columns.tolist()[0:-1] + ['2014-16']
    cur_field = cur_field[cur_field['2014-16'] >= 5]
    cur_field['tech_field'] = key
    cur_field.head()
    df_leaders_per_tech_fileds = pd.concat([df_leaders_per_tech_fileds, cur_field])

Wall time: 995 ms


In [65]:
df_leaders_per_tech_fileds = df_leaders_per_tech_fileds.sort_values(['tech_field', '2014-16'], ascending=[True, False])
df_leaders_per_tech_fileds

,2014,2015,2016,2017,2014-16,tech_field
Kabushiki Kaisha Toshiba,468.0,365.0,308.0,261.0,1141.0,1
Mitsubishi Electric Corporation,484.0,423.0,183.0,80.0,1090.0,1
YAZAKI CORPORATION,306.0,306.0,263.0,231.0,875.0,1
Sony Corporation,249.0,254.0,178.0,105.0,681.0,1
Canon Kabushiki Kaisha,254.0,184.0,164.0,130.0,602.0,1
Toyota Jidosha Kabushiki Kaisha,108.0,226.0,210.0,335.0,544.0,1
Tokyo Electron Limited,200.0,197.0,133.0,143.0,530.0,1
"Nissan Motor Co., Ltd.",195.0,187.0,70.0,10.0,452.0,1
"Honda Motor Co., Ltd.",153.0,126.0,146.0,189.0,425.0,1
"Semiconductor Energy Laboratory Co., Ltd.",137.0,126.0,114.0,92.0,377.0,1


In [66]:
df_leaders_per_tech_fileds.reset_index().to_excel(f'{CITY_NAME}\\{CITY_NAME}_заявители_по_областям.xlsx', index=False)